<a href="https://colab.research.google.com/github/Rashimanish/USA-Flight-Prediction/blob/main/Google%20Colab/Classification_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install joblib

In [ ]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 9.3 MB/s eta 0:00:00


In [ ]:
#import libraries
from google.colab import drive
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import resample
from imblearn.over_sampling import ADASYN
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc, confusion_matrix, classification_report
import catboost as cat
import joblib

In [ ]:
# Mount Google Drive
drive.mount('/content/drive')

# File path
file_path = '/content/drive/My Drive/Cleaned_Data/dataset09.csv'
# Read dataset
df = pd.read_csv(file_path)

Mounted at /content/drive


In [ ]:
df.shape

(1802634, 24)

In [ ]:
pd.set_option('display.max_columns', None)
df.head()

,YEAR,QUARTER,MONTH,DAY_OF_MONTH,FL_DATE,ORIGIN_AIRPORT_ID,ORIGIN,DEST_AIRPORT_ID,DEST,DISTANCE,ORIGIN_CARRIER,CRS_DEP_TIME,DEP_TIME,DEP_DELAY_NEW,DEP_DEL15,CRS_ARR_TIME,ARR_TIME,ARR_DELAY_NEW,ARR_DEL15,Maximum temperature,Minimum temperature,Precipitation,Snowfall,Average wind speed
0,2022,1,1,1,2022-01-01,10397,ATL,11057,CLT,226,DL,645,640.0,0.0,0.0,809,751.0,0.0,0.0,78,68,0.06,0.0,11.86
1,2022,1,1,1,2022-01-01,13204,MCO,13930,ORD,1005,SW,1429,1423.0,0.0,0.0,1639,1611.0,0.0,0.0,84,65,0.00,0.0,7.61
2,2022,1,1,1,2022-01-01,13204,MCO,13930,ORD,1005,SW,1210,1204.0,0.0,0.0,1419,1348.0,0.0,0.0,84,65,0.00,0.0,7.61
3,2022,1,1,1,2022-01-01,13204,MCO,13930,ORD,1005,SW,1117,1109.0,0.0,0.0,1320,1303.0,0.0,0.0,84,65,0.00,0.0,7.61
4,2022,1,1,1,2022-01-01,13204,MCO,13930,ORD,1005,SW,1010,1100.0,50.0,1.0,1200,1307.0,67.0,1.0,84,65,0.00,0.0,7.61


In [ ]:
def drop_cols(df):
    columns_to_drop = [
        'YEAR',
        'QUARTER',
        'DEP_TIME',
        'ARR_TIME',
        'DEP_DEL15',
        'ORIGIN_AIRPORT_ID',
        'DEST_AIRPORT_ID',
        'DEP_DELAY_NEW',
        'ARR_DELAY_NEW',
        'Maximum temperature',
        'Minimum temperature',
        'Precipitation',
        'Snowfall',
        'Average wind speed'
    ]

    df = df.drop(columns=columns_to_drop, errors='ignore')
    return df

In [ ]:
df = drop_cols(df)

In [ ]:
df.head()

,MONTH,DAY_OF_MONTH,FL_DATE,ORIGIN,DEST,DISTANCE,ORIGIN_CARRIER,CRS_DEP_TIME,CRS_ARR_TIME,ARR_DEL15
0,1,1,2022-01-01,ATL,CLT,226,DL,645,809,0.0
1,1,1,2022-01-01,MCO,ORD,1005,SW,1429,1639,0.0
2,1,1,2022-01-01,MCO,ORD,1005,SW,1210,1419,0.0
3,1,1,2022-01-01,MCO,ORD,1005,SW,1117,1320,0.0
4,1,1,2022-01-01,MCO,ORD,1005,SW,1010,1200,1.0


## Feature Engineering Phase

In [ ]:
def feature_engineering(df):
    df['FL_DATE'] = pd.to_datetime(df['FL_DATE'])

    # Extract new features
    df['DAY_OF_WEEK'] = df['FL_DATE'].dt.dayofweek
    df['MONTH'] = df['FL_DATE'].dt.month
    df['HOUR_DEP'] = df['CRS_DEP_TIME'] // 100
    df['MIN_DEP'] = df['CRS_DEP_TIME'] % 100
    df['HOUR_ARR'] = df['CRS_ARR_TIME'] // 100
    df['MIN_ARR'] = df['CRS_ARR_TIME'] % 100
    df['DISTANCE_BIN'] = pd.cut(df['DISTANCE'], bins=[0, 500, 1000, 1500, 2000, 3000], labels=[1, 2, 3, 4, 5])

    # Create interaction features
    df['DISTANCE_DAY_INTERACTION'] = df['DISTANCE'] * df['DAY_OF_WEEK']
    df['HOUR_DEP_ARR_INTERACTION'] = df['HOUR_DEP'] * df['HOUR_ARR']
    df['MONTH_DAY_INTERACTION'] = df['MONTH'] * df['DAY_OF_WEEK']

    # Drop columns no longer needed
    cols_to_drop = ['FL_DATE', 'CRS_DEP_TIME', 'CRS_ARR_TIME']
    df = df.drop(columns=cols_to_drop, errors='ignore')

     # Convert categorical features to numeric using label encoding
    label_encoder = LabelEncoder()
    categorical_columns = ['ORIGIN', 'DEST', 'ORIGIN_CARRIER','DISTANCE_BIN']
    for col in categorical_columns:
        df[col] = label_encoder.fit_transform(df[col])


    return df

In [ ]:
df = feature_engineering(df)

In [ ]:
df.head()

,MONTH,DAY_OF_MONTH,ORIGIN,DEST,DISTANCE,ORIGIN_CARRIER,ARR_DEL15,DAY_OF_WEEK,HOUR_DEP,MIN_DEP,HOUR_ARR,MIN_ARR,DISTANCE_BIN,DISTANCE_DAY_INTERACTION,HOUR_DEP_ARR_INTERACTION,MONTH_DAY_INTERACTION
0,1,1,0,1,226,2,0.0,5,6,45,8,9,0,1130,48,5
1,1,1,9,11,1005,3,0.0,5,14,29,16,39,2,5025,224,5
2,1,1,9,11,1005,3,0.0,5,12,10,14,19,2,5025,168,5
3,1,1,9,11,1005,3,0.0,5,11,17,13,20,2,5025,143,5
4,1,1,9,11,1005,3,1.0,5,10,10,12,0,2,5025,120,5


## **Sampling Using ADASYN**

In [ ]:
def sampling(X, y, seed=42):
    adasyn = ADASYN(random_state=seed)
    X_resampled, y_resampled = adasyn.fit_resample(X, y)

    return X_resampled, y_resampled

In [ ]:
y = df['ARR_DEL15']
X = df.drop(columns=['ARR_DEL15'])

In [ ]:
# Apply the sampling function
seed = 35
X_bal, y_bal = sampling(X, y, seed)

In [ ]:
def evaluate_model(model, X, y):
    metrics_list = []
    fpr_list = []
    tpr_list = []
    roc_auc_list = []

    cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=35)

    for train_index, test_index in cv.split(X, y):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        model.fit(X_train, y_train)

        y_pred = model.predict(X_test)
        y_pred_proba = model.predict_proba(X_test)[:, 1]

        fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
        roc_auc = auc(fpr, tpr)

        metrics = {
            'accuracy': accuracy_score(y_test, y_pred),
            'precision': precision_score(y_test, y_pred),
            'recall': recall_score(y_test, y_pred),
            'f1_score': f1_score(y_test, y_pred),
            'roc_auc': roc_auc,
            'conf_matrix': confusion_matrix(y_test, y_pred)
        }
        metrics_list.append(metrics)
        fpr_list.append(fpr)
        tpr_list.append(tpr)
        roc_auc_list.append(roc_auc)

    avg_metrics = {
        'accuracy': np.mean([m['accuracy'] for m in metrics_list]),
        'precision': np.mean([m['precision'] for m in metrics_list]),
        'recall': np.mean([m['recall'] for m in metrics_list]),
        'f1_score': np.mean([m['f1_score'] for m in metrics_list]),
        'roc_auc': np.mean(roc_auc_list),
        'conf_matrix': np.sum([m['conf_matrix'] for m in metrics_list], axis=0)
    }

    # Average ROC curve values
    mean_fpr = np.linspace(0, 1, 100)
    mean_tpr = np.mean([np.interp(mean_fpr, fpr, tpr) for fpr, tpr in zip(fpr_list, tpr_list)], axis=0)
    mean_tpr[0] = 0.0
    mean_auc = np.mean(roc_auc_list)

    return avg_metrics, mean_fpr, mean_tpr, mean_auc

In [ ]:
def performance_metrics(model_name, metrics):
    metrics_df = pd.DataFrame({
        'Metric': ['Accuracy', 'Precision', 'Recall', 'F1 Score', 'ROC AUC'],
        'Score': [
            metrics['accuracy'],
            metrics['precision'],
            metrics['recall'],
            metrics['f1_score'],
            metrics['roc_auc']
        ]
    })

    print(f"\n{model_name} Model Performance Metrics")
    display(metrics_df)

    conf_matrix_df = pd.DataFrame(metrics['conf_matrix'],
                                  index=['Actual Negative', 'Actual Positive'],
                                  columns=['Predicted Negative', 'Predicted Positive'])

    print("\nConfusion Matrix:")
    display(conf_matrix_df)

# **Cat Boost**

In [ ]:
catboost_model = cat.CatBoostClassifier(
    iterations=1000,
    learning_rate=0.1,
    depth=10,
    l2_leaf_reg=3.0,
    border_count=50,
    subsample=1.0,
    random_seed=42,
    verbose=0
)
avg_metrics_cat, avg_fpr_cat, avg_tpr_cat, avg_roc_auc_cat = evaluate_model(catboost_model, X_bal, y_bal)
performance_metrics("CatBoost", avg_metrics_cat)



CatBoost Model Performance Metrics


,Metric,Score
0,Accuracy,0.810447
1,Precision,0.895437
2,Recall,0.705190
3,F1 Score,0.789007
4,ROC AUC,0.886188



Confusion Matrix:


,Predicted Negative,Predicted Positive
Actual Negative,1267357,115018
Actual Positive,411770,984958


In [ ]:

# Define the Google Drive path
joblib_file = '/content/drive/My Drive/2024_FLIGHT/FLIGHT/tuned_classification_model.pkl'

# Save the model
joblib.dump(catboost_model, joblib_file)
print(f"Model saved to {joblib_file}")




Model saved to /content/drive/My Drive/2024_FLIGHT/FLIGHT/tuned_classification_model.pkl
